In [1]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [4]:
#importing labriries
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [5]:
#disable the use of OpenCL in OpenCV
cv2.ocl.setUseOpenCL(False)

## preparing the train data:

In [6]:
#flow_from_directory isusing for a data like the one i have with diffrent file for each class
train_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1, validation_split=0.2).flow_from_directory(
        'data2/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',subset='training')

Found 23769 images belonging to 7 classes.


## preparing the test data:

In [7]:
val_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1,validation_split=0.2).flow_from_directory(
        'data2/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',subset='validation')


Found 5941 images belonging to 7 classes.


In [8]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=10,
    verbose=1,
    restore_best_weights=True
)

## building the model :

In [9]:

my_mental_health = Sequential([#features extraction phase
layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),#conv2D convolution over images/input with 1 color(greyscale)
layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),#relu is activation function that turn the negative to zero
layers.MaxPooling2D(pool_size=(2, 2)),#maxpooling2d condensing the data (take  the max from the convolution map) to reduice the size
layers.Dropout(0.25),#protect from overfitting
layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
layers.MaxPooling2D(pool_size=(2, 2)),
layers.Dropout(0.25),
layers.Flatten(),#flatten is preparing the data to the neural network
layers.Dense(1024, activation='relu'),#1024 number of nerons
layers.Dropout(0.5),
layers.Dense(7, activation='softmax')])#7units corresponding to the number of classes
#softmax is used to normalize the output and gives probabilities of each class


In [10]:
my_mental_health.compile(loss='categorical_crossentropy', optimizer=Adam( learning_rate=0.0001), metrics=['accuracy'])
#categorical_crossentropy used for multi-class classification 

In [11]:
my_mental_health.summary()# to show the steps of the model for the data

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

## fitting the model :

In [13]:
my_mental_health_info = my_mental_health.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // 64,
        epochs=100,
        validation_data=val_generator,
        validation_steps=val_generator.samples // 64,callbacks=[early_stopping])

Epoch 1/100
371/371 [==============================] - 56s 152ms/step - loss: 1.6893 - accuracy: 0.3242 - val_loss: 1.6573 - val_accuracy: 0.3407
Epoch 2/100
371/371 [==============================] - 58s 157ms/step - loss: 1.6382 - accuracy: 0.3539 - val_loss: 1.6184 - val_accuracy: 0.3635
Epoch 3/100
371/371 [==============================] - 57s 154ms/step - loss: 1.5973 - accuracy: 0.3704 - val_loss: 1.5678 - val_accuracy: 0.3779
Epoch 4/100
371/371 [==============================] - 57s 154ms/step - loss: 1.5561 - accuracy: 0.3905 - val_loss: 1.5233 - val_accuracy: 0.4069
Epoch 5/100
371/371 [==============================] - 57s 152ms/step - loss: 1.5188 - accuracy: 0.4096 - val_loss: 1.4979 - val_accuracy: 0.4175
Epoch 6/100
371/371 [==============================] - 57s 155ms/step - loss: 1.4863 - accuracy: 0.4239 - val_loss: 1.4490 - val_accuracy: 0.4446
Epoch 7/100
371/371 [==============================] - 58s 156ms/step - loss: 1.4525 - accuracy: 0.4462 - val_loss: 1.4303 -

Epoch 57/100
371/371 [==============================] - 55s 147ms/step - loss: 1.0373 - accuracy: 0.6133 - val_loss: 1.0816 - val_accuracy: 0.5941
Epoch 58/100
371/371 [==============================] - 54s 146ms/step - loss: 1.0320 - accuracy: 0.6133 - val_loss: 1.0995 - val_accuracy: 0.5892
Epoch 59/100
371/371 [==============================] - 55s 148ms/step - loss: 1.0231 - accuracy: 0.6146 - val_loss: 1.0866 - val_accuracy: 0.5985
Epoch 60/100
371/371 [==============================] - 54s 147ms/step - loss: 1.0254 - accuracy: 0.6158 - val_loss: 1.0869 - val_accuracy: 0.5939
Epoch 61/100
371/371 [==============================] - 54s 146ms/step - loss: 1.0168 - accuracy: 0.6187 - val_loss: 1.0931 - val_accuracy: 0.5919
Epoch 62/100
371/371 [==============================] - 54s 147ms/step - loss: 1.0111 - accuracy: 0.6222 - val_loss: 1.0906 - val_accuracy: 0.5919
Epoch 63/100
371/371 [==============================] - 55s 149ms/step - loss: 1.0083 - accuracy: 0.6220 - val_loss: 1


## make predictions

In [14]:
test_generator = ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.1,width_shift_range=0.1,height_shift_range=0.1).flow_from_directory(
        'data2/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical', shuffle=False)

Found 7178 images belonging to 7 classes.


In [15]:
test_loss, test_accuracy = my_mental_health.evaluate(test_generator, steps= test_generator.samples// 64)

112/112 [==============================] - 40s 362ms/step - loss: 1.0749 - accuracy: 0.5921


## check accuracy

In [16]:
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Loss: 1.0749
Test Accuracy: 0.5921


In [17]:
predictions = my_mental_health.predict(test_generator)

113/113 [==============================] - 7s 59ms/step


In [18]:
import numpy as np
predicted_labels = np.argmax(predictions, axis=1)  # Get the index of the predicted class with highest probability
true_labels = test_generator.classes  # Get the true labels of the test samples

accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy:", accuracy)

Accuracy: 0.5913903594315966


## save the model 

In [20]:
my_mental_health.save('model.h5')

AttributeError: 'Sequential' object has no attribute 'model'